In [7]:
# Import lib
import os
import numpy as np
import cv2
import imutils

In [8]:
# Set base path
image_path = 'C:/Users/RCV/RCV/week0912_panorama/images'
save_path = 'C:/Users/RCV/RCV/week0912_panorama/panorama_image'

In [25]:
class Panorama:
    def image_stitching(self, images, low_ratio=0.7, homo_Threshold=3.0, percent_Threshold=10, inlier_Threshold=50, is_percent=False, draw_matches=False, pano_draw_match_pair=True):        
        panorama = None
        
        while len(images):
            max_matched_image_index = self.find_max_matched_image(images, homo_Threshold, percent_Threshold, inlier_Threshold, is_percent, draw_matches)
            max_matched_image = images[max_matched_image_index]
            
            if panorama is None:
                panorama = max_matched_image
            else:    
                panorama = self.panorama_stitching(max_matched_image, panorama, pano_draw_match_pair)

                cv2.imshow('panorama', panorama)
                cv2.waitKey(0)

                cv2.destroyAllWindows()

            del images[max_matched_image_index]
            
        return panorama


    def computeSIFT(self, image):
        img2gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        SIFT = cv2.SIFT_create()
        kp, desc = SIFT.detectAndCompute(img2gray, None)

        return kp, desc


    def knn_match(self, desc_A, desc_B):
        BF_matcher = cv2.BFMatcher(cv2.NORM_L1)
        matches = BF_matcher.knnMatch(desc_A, desc_B, k=2)

        return matches


    def valid_match(self, matches, low_ratio):
        valid_matches, valid_matches_idx = [], []
        for m_A, m_B in matches:
            if m_A.distance < m_B.distance * low_ratio:
                valid_matches.append(m_A)
                valid_matches_idx.append((m_A.trainIdx, m_A.queryIdx))

        # print(f'BF matches: {len(matches)}, Good matches: {len(valid_matches)}')
        return valid_matches, valid_matches_idx


    def match_keypoints(self, kp_A, kp_B, desc_A, desc_B, low_ratio):
        possible_matches = self.knn_match(desc_A, desc_B)
        valid_matches, valid_matches_idx = self.valid_match(possible_matches, low_ratio=low_ratio)

        # Coordinate match pairs need at least 4 pairs
        if len(valid_matches) >= 4:
            point_A, point_B = [], []
            for train_idx, query_idx in valid_matches_idx:
                point_A.append(np.float32(kp_A[query_idx].pt))
                point_B.append(np.float32(kp_B[train_idx].pt))

            point_A = np.asarray(point_A).reshape(-1, 1, 2)
            point_B = np.asarray(point_B).reshape(-1, 1, 2)

            return (point_A, point_B, valid_matches, valid_matches_idx)

        else:
            return None
            

    def draw_matches(self, image_A, kp_A, image_B, kp_B, matches):
        draw = cv2.drawMatches(image_A, kp_A, image_B, kp_B, matches, None, flags=2)

        return draw


    # image_A = source, image_B = target
    def find_homography(self, image_A, image_B, max_threshold=3.0):
        # Compute SIFT
        (kp_A, desc_A) = self.computeSIFT(image_A)
        (kp_B, desc_B) = self.computeSIFT(image_B)
            
        # Get valid matches
        matches = self.match_keypoints(kp_A, kp_B, desc_A, desc_B, low_ratio=0.7)

        if matches is None:
            return
            
        (point_A, point_B, valid_match, valid_match_idx) = matches

        # mask has status of inliers and outliers
        H, mask = cv2.findHomography(point_A, point_B, cv2.RANSAC, max_threshold)
        matched_mask = mask.ravel().tolist()

        return H, matched_mask


    def find_matches_percent(self, matched_mask):
        outlier, inlier = 0, 0

        for mask in matched_mask:
            if mask:
                inlier += 1
            else:
                outlier += 1

        percent = ((inlier) / (inlier + outlier)) * 100
        # print(f'inlier: {inlier} / outlier: {outlier}')
        
        return percent

    
    def find_inlier_matches(self, matched_mask):
        inlier = 0

        for mask in matched_mask:
            if mask:
                inlier += 1

        return inlier


    def find_max_matched_with_pano(self, pano, image_list, homo_Threshold, percent_Threshold, inlier_Threshold, is_percent, draw_matches):
        num_of_images = len(image_list)
        max_matched_index, max_count = 0, 0

        if num_of_images == 1:
            return max_matched_index

        for image_index, image in enumerate(image_list):
            match_count = 0
            if draw_matches:
                # Compute SIFT
                (kp_i, desc_i) = self.computeSIFT(image)
                (kp_p, desc_p) = self.computeSIFT(pano)
            
                # Get valid matches
                matches = self.match_keypoints(kp_i, kp_p, desc_i, desc_p, low_ratio=0.7)
            
                if matches is None:
                    continue
            
                (point_A, point_B, valid_match, valid_match_idx) = matches   

                draw = self.draw_matches(image, kp_i, pano, kp_p, valid_match)
                
                cv2.imshow('Match draw', draw)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

            # Compute homography, and Get perspective of image
            _, matched_mask = self.find_homography(pano, image, homo_Threshold)
            
            if is_percent:
                percent = self.find_matches_percent(matched_mask)
                # print(f'A image index: {image_A_index} <-> {image_B_index}: B image index | matching percent: {percent}')

                if percent > percent_Threshold: 
                    match_count += 1

                if percent < 10: break

            else:
                num_of_inliers = self.find_inlier_matches(matched_mask)
                # print(f'A image index: {image_A_index} <-> {image_B_index}: B image index | number of inliers : {num_of_inliers}')

                if num_of_inliers > inlier_Threshold:
                    match_count += 1

            if max_count < match_count:
                max_match_index = image_index
                max_count = match_count 

        return max_match_index
                    
    
    def find_max_matched_image(self, image_list, homo_Threshold, percent_Threshold, inlier_Threshold, is_percent, draw_matches):
        num_of_images = len(image_list)
        max_match_index, max_count = 0, 0

        if num_of_images == 1:
            return max_match_index

        print(f'number of images: {num_of_images}')

        for image_A_index in range(num_of_images):
            match_count = 0
            for image_B_index in range(num_of_images):
                if image_A_index != image_B_index:
                    if draw_matches:
                        # Compute SIFT
                        (kp_A, desc_A) = self.computeSIFT(image_list[image_A_index])
                        (kp_B, desc_B) = self.computeSIFT(image_list[image_B_index])
                    
                        # Get valid matches
                        matches = self.match_keypoints(kp_A, kp_B, desc_A, desc_B, low_ratio=0.7)
                    
                        if matches is None:
                            continue
                    
                        (point_A, point_B, valid_match, valid_match_idx) = matches   

                        draw = self.draw_matches(image_list[image_A_index], kp_A, image_list[image_B_index], kp_B, valid_match)
                        
                        cv2.imshow('Match draw', draw)
                        cv2.waitKey(0)
                        cv2.destroyAllWindows()

                    # Compute homography, and Get perspective of image
                    _, matched_mask = self.find_homography(image_list[image_A_index], image_list[image_B_index], homo_Threshold)
                    
                    if is_percent:
                        percent = self.find_matches_percent(matched_mask)
                        # print(f'A image index: {image_A_index} <-> {image_B_index}: B image index | matching percent: {percent}')

                        if percent > percent_Threshold:
                            match_count += 1

                        if percent < 10: break

                    else:
                        num_of_inliers = self.find_inlier_matches(matched_mask)
                        # print(f'A image index: {image_A_index} <-> {image_B_index}: B image index | number of inliers : {num_of_inliers}')

                        if num_of_inliers > inlier_Threshold:
                            match_count += 1

                if max_count < match_count:
                    max_match_index = image_A_index
                    max_count = match_count 

            # print(f'index:{image_A_index} image be matching {match_count} images')

        # print(f'max matched image index: {max_match_index}, count: {max_count}')

        return max_match_index


    def warp_perspective(self, src, dst, H):
        warpped_image = cv2.warpPerspective(dst, H, ((src.shape[1] + dst.shape[1]), dst.shape[0]))

        return warpped_image
        
    
    def panorama_stitching(self, refer, pano, pano_draw_match_pair):
        if pano_draw_match_pair:
            # Compute SIFT
            (kp_A, desc_A) = self.computeSIFT(refer)
            (kp_B, desc_B) = self.computeSIFT(pano)
        
            # Get valid matches
            matches = self.match_keypoints(kp_A, kp_B, desc_A, desc_B, low_ratio=0.7)
        
            if matches is None:
                pass
        
            (point_A, point_B, valid_match, valid_match_idx) = matches   

            draw = self.draw_matches(refer, kp_A, pano, kp_B, valid_match)
            
            cv2.imshow('Match draw', draw)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        h_r, w_r, _ = refer.shape
        h_p, w_p, _ = pano.shape

        H, _ = self.find_homography(pano, refer)

        points = np.float32([[0, 0], [0, h_r - 1], [h_r - 1, h_r - 1], [h_r - 1, 0]]).reshape(-1, 1, 2)

        P_Trans_corner_points = cv2.perspectiveTransform(points, H)

        min_x1 = min(P_Trans_corner_points[0][0][0], P_Trans_corner_points[1][0][0])
        min_x2 = min(P_Trans_corner_points[2][0][0], P_Trans_corner_points[3][0][0])
        min_y1 = min(P_Trans_corner_points[0][0][1], P_Trans_corner_points[1][0][1])
        min_y2 = min(P_Trans_corner_points[2][0][1], P_Trans_corner_points[3][0][1])

        max_x1 = max(P_Trans_corner_points[0][0][0], P_Trans_corner_points[1][0][0])
        max_x2 = max(P_Trans_corner_points[2][0][0], P_Trans_corner_points[3][0][0])
        max_y1 = max(P_Trans_corner_points[0][0][1], P_Trans_corner_points[1][0][1])
        max_y2 = max(P_Trans_corner_points[2][0][1], P_Trans_corner_points[3][0][1])

        min_x = min(min_x1, min_x2)
        min_y = min(min_y1, min_y2)
        max_x = max(max_x1, max_x2)
        max_y = max(max_y1, max_y2)

        # Transformation matrix
        T_matrix = np.eye(3)
        
        if min_x < 0:
            max_x = w_p - min_x
            T_matrix[0][2] = -min_x
        else:
            if max_x < w_p:
                max_x = w_p

        if min_y < 0:
            max_y = h_p - min_y
            T_matrix[1][2] = -min_y
        else:
            if max_y < h_p:
                max_y = h_p
        
        warpped_refer = cv2.warpPerspective(refer, T_matrix, (int(max_y), int(max_x)), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
        panorama = cv2.warpPerspective(pano, np.dot(T_matrix, H), (int(max_y), int(max_x)), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=0)

        warpped_refer_sum = warpped_refer.sum(axis=-1)
        h_p1, w_p1 = warpped_refer_sum.shape

        for h in range(h_p1 - 1):
            for w in range(w_p1 - 1):
                if warpped_refer_sum[h, w] > 0:
                    panorama[h, w, :] = warpped_refer[h, w, :]

        # panorama[0:warpped_refer.shape[0], 0:warpped_refer.shape[1]] = warpped_refer

        return panorama
    

    def panorama_auto_stitching(self, pano, refer):
        pano = cv2.resize(pano, (600, 450))
        pano = cv2.resize(refer, (600, 450))
        
        H, _ = self.find_homography(pano, refer)

        warpped = self.warp_perspective(pano, refer, H)
        warpped[0:refer.shape[0], 0:refer.shape[1]] = refer

        # Audo Stitcher
        stitcher = cv2.Stitcher.create(cv2.Stitcher_PANORAMA)
        _, pano = stitcher.stitch(pano, warpped)
        
        return pano

In [26]:
if __name__ == "__main__":
    image_list = os.listdir(image_path)

    images = []
    for image in image_list:
        images.append(cv2.imread(os.path.join(image_path, image)))

    panorama = Panorama()
    
    pano_image = panorama.image_stitching(images)

    # cv2.imwrite(os.path.join(save_path, 'pano_result.jpg'), pano_image)
    # cv2.imshow('panorama image', pano_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

number of images: 8
number of images: 7
number of images: 6
number of images: 5


KeyboardInterrupt: 